Dependencies

tokenizers
torch
transformers
First we load our RobBERT model that was pretrained. We also load in RobBERT's tokenizer.

Because we only want to get results, we have to disable dropout etc. So we add model.eval().

Note: we pretrained both RobBERT v1 and RobBERT v2 in Fairseq and converted these checkpoints to HuggingFace. The MLM task behaves a bit differently.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 14.4 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 3.3 MB 63.7 MB/s 
     |████████████████████████████████| 636 kB 62.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
from transformers import RobertaTokenizer, AutoModelForSequenceClassification, AutoConfig

from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch
tokenizer = RobertaTokenizer.from_pretrained('pdelobelle/robbert-v2-dutch-base')
model = RobertaForMaskedLM.from_pretrained('pdelobelle/robbert-v2-dutch-base', return_dict=True)
model = model.to( 'cuda' if torch.cuda.is_available() else 'cpu' )
model.eval()
#model = RobertaForMaskedLM.from_pretrained('pdelobelle/robbert-v2-dutch-base', return_dict=True)
print("RobBERT model loaded")


RobBERT model loaded


In [3]:

sequence = f"Er staat een {tokenizer.mask_token} in mijn tuin."

In [4]:
input = tokenizer.encode(sequence, return_tensors="pt").to( 'cuda' if torch.cuda.is_available() else 'cpu' )
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

Now that we have our tokenized input and the position of the masked token, we pass the input through RobBERT.

This will give us a predicting for all tokens, but we're only interested in the <mask> token.

In [5]:
with torch.no_grad():
    token_logits = model(input).logits

In [6]:

logits = token_logits[0, mask_token_index, :].squeeze()
prob = logits.softmax(dim=0)
values, indeces = prob.topk(k=15, dim=0)

for index, token in enumerate(tokenizer.convert_ids_to_tokens(indeces)):
    print(f"{token:20} | id = {indeces[index]:4} | p = {values[index]}")

Ġboom                | id = 2600 | p = 0.16917362809181213
Ġbankje              | id = 21620 | p = 0.08176498115062714
Ġschutting           | id = 15000 | p = 0.03842025250196457
Ġvijver              | id = 8217 | p = 0.03808601573109627
Ġplant               | id = 2721 | p = 0.03249496594071388
Ġfontein             | id = 23425 | p = 0.027768243104219437
Ġhuis                | id =  251 | p = 0.023765120655298233
Ġfoto                | id =  290 | p = 0.022633107379078865
Ġkunstwerk           | id = 8925 | p = 0.02188107743859291
Ġlamp                | id = 3737 | p = 0.016486817970871925
Ġhek                 | id = 9746 | p = 0.013344597071409225
Ġpaal                | id = 8626 | p = 0.012663916684687138
Ġraam                | id = 2973 | p = 0.01257301401346922
Ġhuisje              | id = 5524 | p = 0.012214199639856815
Ġtuinhuis            | id = 25124 | p = 0.011169261299073696



RobBERT with pipelines

We can also use the fill-mask pipeline from Huggingface, that does basically the same thing.

In [7]:
from transformers import pipeline
p = pipeline("fill-mask", model="pdelobelle/robbert-v2-dutch-base")

In [8]:
p(sequence)

[{'score': 0.16917362809181213,
  'sequence': 'Er staat een boom in mijn tuin.',
  'token': 2600,
  'token_str': ' boom'},
 {'score': 0.08176498115062714,
  'sequence': 'Er staat een bankje in mijn tuin.',
  'token': 21620,
  'token_str': ' bankje'},
 {'score': 0.03842025250196457,
  'sequence': 'Er staat een schutting in mijn tuin.',
  'token': 15000,
  'token_str': ' schutting'},
 {'score': 0.03808601573109627,
  'sequence': 'Er staat een vijver in mijn tuin.',
  'token': 8217,
  'token_str': ' vijver'},
 {'score': 0.03249496594071388,
  'sequence': 'Er staat een plant in mijn tuin.',
  'token': 2721,
  'token_str': ' plant'}]